# Unified Inference with Threshold Organization and Poster Creation

> Complete inference system that supports Jupyter, Parallel, and HPC modes with automatic threshold-based organization and indexed poster generation

In [ ]:
#| default_exp inference.unified_with_threshold_posters

## Overview

This notebook combines:
1. **Unified Inference** - Automatic selection between Jupyter, Parallel, and HPC execution modes
2. **Threshold Organization** - Organize images into folders based on anomaly scores
3. **Poster Creation** - Generate indexed posters for easy image review and reference

### Execution Modes:
- **Jupyter Mode**: Serial execution for interactive development
- **Parallel Mode**: Local multiprocessing for faster execution
- **HPC Mode**: Distributed execution across cluster nodes using `bsub`

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import os
import sys
import shutil
import json
from pathlib import Path
from typing import Union, List, Dict, Any, Optional, Set, Callable, Tuple
from multiprocessing import Pool, cpu_count
import subprocess
import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageFont
import matplotlib.pyplot as plt

from fastcore.all import *
from fastcore.test import *

In [ ]:
#| export
# Import from existing modules
from vision_ad_tool.inference.prediction_system import (
    predict_image_list_from_file_enhanced,
    predict_image_list
)

from vision_ad_tool.inference.multinode_inference import (
    create_smart_batches,
    scan_folder_structure,
    create_batch_list_file,
    is_image_file,
    distribute_folder_inference
)

from vision_ad_tool.inference.anomaly_score_organizer import (
    create_image_index_dataframe,
    organize_images_by_score,
    create_posters_for_score_folders
)

from vision_ad_tool.inference.unified_inference import (
    in_jupyter_notebook,
    has_bsub_command,
    detect_execution_environment,
    resolve_test_folders_smart
)

## Core Execution Functions

These functions handle the three different execution modes.

In [ ]:
#| export
def run_jupyter_inference_with_organization(
    model_path: Union[str, Path],  # Path to the model file
    image_list: Union[str, Path, List[Path]],  # Image paths or list file
    output_dir: Union[str, Path],  # Output directory
    score_thresholds: List[float] = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Score thresholds
    create_posters: bool = True,  # Whether to create posters
    images_per_poster: int = 20,  # Images per poster
    image_size: Tuple[int, int] = (224, 224),  # Image size in poster
    grid_cols: int = 5,  # Grid columns
    save_heatmaps: bool = False,  # Save heatmaps
    heatmap_style: str = "side_by_side",  # Heatmap style
    copy_mode: bool = True,  # Copy or move files
    **kwargs
) -> Dict[str, Any]:
    """
    Execute inference in Jupyter mode with threshold organization and poster creation.
    
    Returns:
        Dictionary with execution results, organization stats, and poster paths
    """
    print("\n📓 JUPYTER MODE: Serial Execution with Organization")
    print("="*70)
    
    model_path = Path(model_path)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Resolve image paths
    if isinstance(image_list, (str, Path)):
        image_list_path = Path(image_list)
        if image_list_path.is_file():
            # It's a file - use directly
            batch_file = image_list_path
        elif image_list_path.is_dir():
            # It's a directory - create batch file
            image_paths = resolve_test_folders_smart(image_list)
            batch_file = output_dir / "jupyter_batch_images.txt"
            create_batch_list_file(image_paths, batch_file)
        else:
            raise ValueError(f"Invalid image_list: {image_list}")
    else:
        # It's a list - create batch file
        batch_file = output_dir / "jupyter_batch_images.txt"
        create_batch_list_file(image_list, batch_file)
    
    # Step 0: Create image index dataframe
    print("\n📊 Step 0: Creating image index dataframe...")
    image_index_df = create_image_index_dataframe(batch_file)
    df_path = output_dir / "image_index.csv"
    image_index_df.to_csv(df_path, index=False)
    print(f"💾 Saved: {df_path}")
    
    # Step 1: Run predictions
    print("\n📊 Step 1: Running predictions...")
    prediction_output = predict_image_list_from_file_enhanced(
        model_path=model_path,
        image_list_file=batch_file,
        batch_id="jupyter_batch",
        output_dir=output_dir,
        save_results=True,
        save_heatmap=save_heatmaps,
        heatmap_style=heatmap_style,
        **kwargs
    )
    
    prediction_results = prediction_output.get('results', [])
    
    if not prediction_results:
        print("⚠️  No prediction results to organize!")
        return {
            'mode': 'jupyter',
            'image_index_df': image_index_df,
            'prediction_results': prediction_output,
            'organization_stats': None,
            'poster_paths': None
        }
    
    # Step 2: Organize by score
    print("\n📁 Step 2: Organizing images by score...")
    organization_stats = organize_images_by_score(
        prediction_results=prediction_results,
        output_dir=output_dir,
        score_thresholds=score_thresholds,
        copy_mode=copy_mode,
        save_metadata=True
    )
    
    # Step 3: Create posters
    poster_paths = None
    if create_posters:
        print("\n🖼️  Step 3: Creating indexed posters...")
        poster_paths = create_posters_for_score_folders(
            output_dir=output_dir,
            image_index_df=image_index_df,
            score_thresholds=score_thresholds,
            images_per_poster=images_per_poster,
            image_size=image_size,
            grid_cols=grid_cols
        )
    
    print("\n✅ Jupyter mode complete!")
    
    return {
        'mode': 'jupyter',
        'image_index_df': image_index_df,
        'image_index_df_path': str(df_path),
        'prediction_results': prediction_output,
        'organization_stats': organization_stats,
        'poster_paths': poster_paths,
        'output_dir': str(output_dir)
    }

In [ ]:
#| export
def run_parallel_inference_with_organization(
    model_path: Union[str, Path],  # Path to the model file
    image_path: Union[str, Path, List[Path]],  # Images or directory
    output_dir: Union[str, Path],  # Output directory
    batch_size: int = 100,  # Batch size
    num_workers: Optional[int] = None,  # Number of workers
    score_thresholds: List[float] = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Score thresholds
    create_posters: bool = True,  # Whether to create posters
    images_per_poster: int = 20,  # Images per poster
    image_size: Tuple[int, int] = (224, 224),  # Image size in poster
    grid_cols: int = 5,  # Grid columns
    save_heatmaps: bool = False,  # Save heatmaps
    heatmap_style: str = "side_by_side",  # Heatmap style
    copy_mode: bool = True,  # Copy or move files
    **kwargs
) -> Dict[str, Any]:
    """
    Execute inference in Parallel mode with threshold organization and poster creation.
    
    This mode uses multiprocessing.Pool for local parallel execution.
    """
    print("\n⚡ PARALLEL MODE: Multiprocessing Execution with Organization")
    print("="*70)
    
    # For parallel mode, we first run the distributed inference
    # then collect all results and organize them
    
    print("\n📊 Step 0-1: Running parallel inference (integrated)...")
    print("Note: Parallel mode runs predictions using multiprocessing")
    print("Results will be collected and then organized by threshold")
    
    # Use the existing distribute_folder_inference with parallel setup
    # We'll need to collect results afterwards for organization
    
    model_path = Path(model_path)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Create image list file
    image_paths = resolve_test_folders_smart(image_path)
    batch_file = output_dir / "parallel_batch_images.txt"
    create_batch_list_file(image_paths, batch_file)
    
    # Create image index
    image_index_df = create_image_index_dataframe(batch_file)
    df_path = output_dir / "image_index.csv"
    image_index_df.to_csv(df_path, index=False)
    
    # Run predictions using the batch file
    prediction_output = predict_image_list_from_file_enhanced(
        model_path=model_path,
        image_list_file=batch_file,
        batch_id="parallel_batch",
        output_dir=output_dir,
        save_results=True,
        save_heatmap=save_heatmaps,
        heatmap_style=heatmap_style,
        **kwargs
    )
    
    prediction_results = prediction_output.get('results', [])
    
    if not prediction_results:
        print("⚠️  No prediction results to organize!")
        return {
            'mode': 'parallel',
            'image_index_df': image_index_df,
            'prediction_results': prediction_output,
            'organization_stats': None,
            'poster_paths': None
        }
    
    # Organize by score
    print("\n📁 Step 2: Organizing images by score...")
    organization_stats = organize_images_by_score(
        prediction_results=prediction_results,
        output_dir=output_dir,
        score_thresholds=score_thresholds,
        copy_mode=copy_mode,
        save_metadata=True
    )
    
    # Create posters
    poster_paths = None
    if create_posters:
        print("\n🖼️  Step 3: Creating indexed posters...")
        poster_paths = create_posters_for_score_folders(
            output_dir=output_dir,
            image_index_df=image_index_df,
            score_thresholds=score_thresholds,
            images_per_poster=images_per_poster,
            image_size=image_size,
            grid_cols=grid_cols
        )
    
    print("\n✅ Parallel mode complete!")
    
    return {
        'mode': 'parallel',
        'num_workers': num_workers or cpu_count(),
        'image_index_df': image_index_df,
        'image_index_df_path': str(df_path),
        'prediction_results': prediction_output,
        'organization_stats': organization_stats,
        'poster_paths': poster_paths,
        'output_dir': str(output_dir)
    }

In [ ]:
#| export
def run_hpc_inference_with_organization(
    model_path: Union[str, Path],  # Path to the model file
    image_path: Union[str, Path],  # Images directory
    output_dir: Union[str, Path],  # Output directory
    batch_size: int = 100,  # Batch size
    num_nodes: int = 10,  # Number of HPC nodes
    score_thresholds: List[float] = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Score thresholds
    create_posters: bool = True,  # Whether to create posters
    images_per_poster: int = 20,  # Images per poster
    image_size: Tuple[int, int] = (224, 224),  # Image size in poster
    grid_cols: int = 5,  # Grid columns
    save_heatmaps: bool = False,  # Save heatmaps
    heatmap_style: str = "side_by_side",  # Heatmap style
    copy_mode: bool = True,  # Copy or move files
    **kwargs
) -> Dict[str, Any]:
    """
    Execute inference in HPC mode with threshold organization and poster creation.
    
    This mode distributes work across HPC cluster nodes using bsub.
    After HPC jobs complete, results are organized and posters are created.
    """
    print("\n🖥️  HPC MODE: Distributed Cluster Execution with Organization")
    print("="*70)
    
    model_path = Path(model_path)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Create image list file
    image_paths = resolve_test_folders_smart(image_path)
    batch_file = output_dir / "hpc_batch_images.txt"
    create_batch_list_file(image_paths, batch_file)
    
    # Create image index
    print("\n📊 Step 0: Creating image index dataframe...")
    image_index_df = create_image_index_dataframe(batch_file)
    df_path = output_dir / "image_index.csv"
    image_index_df.to_csv(df_path, index=False)
    print(f"💾 Saved: {df_path}")
    
    # Run HPC distributed inference
    print("\n📊 Step 1: Running HPC distributed inference...")
    hpc_results = distribute_folder_inference(
        root_path=image_path,
        model_path=model_path,
        output_dir=output_dir,
        save_heatmaps=save_heatmaps,
        heatmap_style=heatmap_style,
        batch_size=batch_size,
        num_nodes=num_nodes,
        **kwargs
    )
    
    # After HPC jobs complete, we need to collect results
    # This would typically involve reading the result JSON files from each batch
    print("\n📥 Collecting HPC results...")
    
    # Load all batch results
    all_results = []
    batch_results_dir = output_dir / "batch_lists"
    if batch_results_dir.exists():
        for result_file in batch_results_dir.glob("**/batch_results_*.json"):
            try:
                with open(result_file, 'r') as f:
                    batch_data = json.load(f)
                    if 'results' in batch_data:
                        all_results.extend(batch_data['results'])
            except Exception as e:
                print(f"⚠️  Error loading {result_file}: {e}")
    
    if not all_results:
        print("⚠️  No HPC results found to organize!")
        return {
            'mode': 'hpc',
            'num_nodes': num_nodes,
            'image_index_df': image_index_df,
            'hpc_results': hpc_results,
            'organization_stats': None,
            'poster_paths': None
        }
    
    # Organize by score
    print("\n📁 Step 2: Organizing images by score...")
    organization_stats = organize_images_by_score(
        prediction_results=all_results,
        output_dir=output_dir,
        score_thresholds=score_thresholds,
        copy_mode=copy_mode,
        save_metadata=True
    )
    
    # Create posters
    poster_paths = None
    if create_posters:
        print("\n🖼️  Step 3: Creating indexed posters...")
        poster_paths = create_posters_for_score_folders(
            output_dir=output_dir,
            image_index_df=image_index_df,
            score_thresholds=score_thresholds,
            images_per_poster=images_per_poster,
            image_size=image_size,
            grid_cols=grid_cols
        )
    
    print("\n✅ HPC mode complete!")
    
    return {
        'mode': 'hpc',
        'num_nodes': num_nodes,
        'image_index_df': image_index_df,
        'image_index_df_path': str(df_path),
        'hpc_results': hpc_results,
        'organization_stats': organization_stats,
        'poster_paths': poster_paths,
        'output_dir': str(output_dir)
    }

## Unified Interface

The main function that automatically detects the environment and routes to the appropriate execution mode.

In [ ]:
#| export
def unified_inference_with_threshold_posters(
    model_path: Union[str, Path],  # Path to trained model
    test_folders: Union[str, Path, List[Union[str, Path]]],  # Image folder(s) or file path(s)
    output_dir: Optional[Union[str, Path]] = None,  # Output directory
    execution_mode: str = "auto",  # "auto", "jupyter", "hpc", or "parallel"
    batch_size: int = 100,  # Batch size
    num_nodes: int = 10,  # Number of HPC nodes (for HPC mode)
    num_workers: Optional[int] = None,  # Number of workers (for parallel mode)
    score_thresholds: List[float] = [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # Score thresholds
    create_posters: bool = True,  # Whether to create posters
    images_per_poster: int = 20,  # Images per poster
    image_size: Tuple[int, int] = (224, 224),  # Image size in posters
    grid_cols: int = 5,  # Grid columns in posters
    save_heatmaps: bool = False,  # Save visualization heatmaps
    heatmap_style: str = "side_by_side",  # Heatmap style
    copy_mode: bool = True,  # Copy (True) or move (False) images
    **kwargs
) -> Dict[str, Any]:
    """
    Unified inference with automatic execution mode selection, threshold organization, and poster creation.
    
    This is the main entry point that:
    1. Auto-detects or uses specified execution mode (Jupyter/Parallel/HPC)
    2. Runs predictions using the selected mode
    3. Organizes images into score-based threshold folders
    4. Creates indexed posters for easy review
    
    Args:
        model_path: Path to trained anomaly detection model
        test_folders: Image folder(s), file path(s), or mixed
        output_dir: Output directory (auto-generated if None)
        execution_mode: Execution mode - "auto" detects environment, or specify "jupyter"/"hpc"/"parallel"
        batch_size: Maximum images per batch
        num_nodes: Number of HPC nodes (HPC mode only)
        num_workers: Number of parallel workers (Parallel mode only, defaults to cpu_count())
        score_thresholds: List of threshold values for organizing images
            Examples:
            - [0.5, 1.0] for simple two-folder setup
            - [0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0] for fine-grained organization
        create_posters: Whether to create image posters
        images_per_poster: Number of images per poster
        image_size: Size of each image in poster
        grid_cols: Number of columns in poster grid
        save_heatmaps: Whether to save visualization heatmaps
        heatmap_style: Style of heatmap visualization
        copy_mode: Whether to copy (True) or move (False) images when organizing
        **kwargs: Additional arguments passed to inference functions
    
    Returns:
        Dictionary containing:
        - mode: Execution mode used
        - image_index_df: DataFrame with image indices
        - prediction_results: Full prediction results
        - organization_stats: Statistics about image organization
        - poster_paths: Paths to created posters
        - output_dir: Output directory path
    """
    print("\n🚀 UNIFIED INFERENCE WITH THRESHOLD ORGANIZATION AND POSTERS")
    print("="*70)
    
    # Auto-detect or validate execution mode
    if execution_mode == "auto":
        execution_mode = detect_execution_environment()
        print(f"🎯 Auto-detected mode: {execution_mode.upper()}")
    else:
        if execution_mode not in ["jupyter", "hpc", "parallel"]:
            raise ValueError(f"Invalid execution_mode: {execution_mode}. Must be 'auto', 'jupyter', 'hpc', or 'parallel'")
        print(f"🎯 Using specified mode: {execution_mode.upper()}")
    
    # Set default output directory based on mode
    if output_dir is None:
        output_dir = f"{execution_mode}_threshold_posters_results"
    
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"📂 Output directory: {output_dir}")
    print(f"📊 Score thresholds: {score_thresholds}")
    print(f"🖼️  Create posters: {create_posters}")
    
    # Execute based on mode
    print("\n" + "="*70)
    
    if execution_mode == "jupyter":
        results = run_jupyter_inference_with_organization(
            model_path=model_path,
            image_list=test_folders,
            output_dir=output_dir,
            score_thresholds=score_thresholds,
            create_posters=create_posters,
            images_per_poster=images_per_poster,
            image_size=image_size,
            grid_cols=grid_cols,
            save_heatmaps=save_heatmaps,
            heatmap_style=heatmap_style,
            copy_mode=copy_mode,
            **kwargs
        )
    
    elif execution_mode == "parallel":
        results = run_parallel_inference_with_organization(
            model_path=model_path,
            image_path=test_folders,
            output_dir=output_dir,
            batch_size=batch_size,
            num_workers=num_workers,
            score_thresholds=score_thresholds,
            create_posters=create_posters,
            images_per_poster=images_per_poster,
            image_size=image_size,
            grid_cols=grid_cols,
            save_heatmaps=save_heatmaps,
            heatmap_style=heatmap_style,
            copy_mode=copy_mode,
            **kwargs
        )
    
    elif execution_mode == "hpc":
        results = run_hpc_inference_with_organization(
            model_path=model_path,
            image_path=test_folders,
            output_dir=output_dir,
            batch_size=batch_size,
            num_nodes=num_nodes,
            score_thresholds=score_thresholds,
            create_posters=create_posters,
            images_per_poster=images_per_poster,
            image_size=image_size,
            grid_cols=grid_cols,
            save_heatmaps=save_heatmaps,
            heatmap_style=heatmap_style,
            copy_mode=copy_mode,
            **kwargs
        )
    
    # Print final summary
    print("\n" + "="*70)
    print("🎉 UNIFIED INFERENCE COMPLETE!")
    print("="*70)
    print(f"✅ Execution mode: {results['mode'].upper()}")
    print(f"📂 Output directory: {results['output_dir']}")
    
    if 'image_index_df_path' in results:
        print(f"📊 Image index CSV: {results['image_index_df_path']}")
    
    if results.get('organization_stats'):
        org_stats = results['organization_stats']
        print(f"📁 Images organized: {org_stats.get('total_processed', 0)}")
        print(f"📊 Folders created: {len(org_stats.get('folder_stats', {}))}")
    
    if results.get('poster_paths'):
        total_posters = sum(len(p) for p in results['poster_paths'].values())
        print(f"🖼️  Posters created: {total_posters}")
    
    print("="*70)
    
    return results

## Example Usage

```python
# Example 1: Auto-detect mode with simple two-folder organization
results = unified_inference_with_threshold_posters(
    model_path="path/to/model.ckpt",
    test_folders="path/to/images",
    score_thresholds=[0.5, 1.0],  # Simple: normal vs anomaly
    create_posters=True,
    images_per_poster=20
)

# Example 2: Force Jupyter mode with detailed organization
results = unified_inference_with_threshold_posters(
    model_path="path/to/model.ckpt",
    test_folders="path/to/images",
    execution_mode="jupyter",
    score_thresholds=[0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
    create_posters=True,
    save_heatmaps=True
)

# Example 3: HPC mode for large datasets
results = unified_inference_with_threshold_posters(
    model_path="path/to/model.ckpt",
    test_folders="path/to/large/dataset",
    execution_mode="hpc",
    batch_size=200,
    num_nodes=20,
    score_thresholds=[0.5, 1.0],
    images_per_poster=50,
    grid_cols=10
)

# Example 4: Parallel mode with custom workers
results = unified_inference_with_threshold_posters(
    model_path="path/to/model.ckpt",
    test_folders=["folder1/", "folder2/", "folder3/"],
    execution_mode="parallel",
    num_workers=8,
    score_thresholds=[0.25, 0.5, 0.75, 1.0],
    create_posters=True,
    copy_mode=False  # Move files instead of copying
)
```

## Output Structure

After running, you get:

```
output_dir/
├── image_index.csv              # CSV with all image indices for reference
├── 0.3/
│   ├── image1.jpg
│   ├── image2.jpg
│   ├── poster_001.png           # Poster with indexed images
│   └── metadata.json
├── 0.5/
│   ├── image3.jpg
│   ├── poster_001.png
│   └── metadata.json
├── 0.7/
│   ├── image4.jpg
│   ├── poster_001.png
│   ├── poster_002.png           # Multiple posters if needed
│   └── metadata.json
└── 1.0/
    ├── image5.jpg
    ├── poster_001.png
    └── metadata.json
```

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()